In [15]:
import pandas as pd


In [ ]:

# Load case_file.csv (assuming it has a column named 'serial_no')
case_file = pd.read_csv("../dataset/preprocessed/case_file_cleaned.csv", usecols=["serial_no"])

# Load statement.csv in chunks to handle large file efficiently
statement_chunks = pd.read_csv("../dataset/preprocessed/statement.csv", chunksize=1000000)

# Convert serial_no column in case_file to a set for fast lookup
valid_serials = set(case_file["serial_no"])

classification_file = pd.read_csv("../dataset/preprocessed/classification.csv")

# Create a new CSV file to store the cleaned data
output_file = "../dataset/processed/statement_cleaned.csv"


C:\Users\vtngu\AppData\Local\Temp\ipykernel_45828\85597204.py:10: DtypeWarning: Columns (0,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  classification_file = pd.read_csv("../dataset/preprocessed/classification.csv")


Cleaning complete. Saved filtered data to ../dataset/processed/statement_cleaned.csv.


In [ ]:

# Process and filter in chunks
with open(output_file, "w", newline="", encoding="utf-8") as f:
    for chunk in statement_chunks:
        filtered_chunk = chunk[chunk["serial_no"].isin(valid_serials)]
        filtered_chunk.to_csv(f, index=False, header=f.tell()==0)  # Write header only once

print(f"Cleaning complete. Saved filtered data to {output_file}.")


In [17]:
statement_cleaned = pd.read_csv(output_file)
pm_count = statement_cleaned[statement_cleaned['statement_type_cd'].str.startswith('PM')].shape[0]
print(f"Number of entries with Pseudo Mark: {pm_count}")

Number of entries with Pseudo Mark: 112


In [18]:
pm_entries = statement_cleaned[statement_cleaned['statement_type_cd'].str.startswith('PM')]
print(pm_entries)

     statement_type_cd                                     statement_text  \
18              PM0001                               WASHBURNS GOLD MEDAL   
54              PM0001               SCOTTS EMULSION PPP OF PURE COD LIVE   
67              PM0001                              WOODWARDS CRIPE WATER   
68              PM0001                             THE DAINTY 1ST QUALITY   
71              PM0001  NELSON MORRIS AND COMPANY BRAND EXTRA SUGAR CU...   
...                ...                                                ...   
1394            PM0001                                           JB SMITH   
1401            PM0001                                         GH BARNETT   
1403            PM0001                                                 KF   
1419            PM0001                                                 KF   
1422            PM0001                                            KISS-ME   

      serial_no  
18     70013253  
54     70019225  
67     70019979  
68 

In [19]:
# Load case_file_cleaned.csv (assuming it has columns named 'serial_no' and 'mark_id_char')
case_file_cleaned = pd.read_csv("../dataset/preprocessed/case_file_cleaned.csv", usecols=["serial_no", "mark_id_char"])

# Save the extracted data to mark_cleaned.csv
case_file_cleaned.to_csv("../dataset/processed/mark_cleaned.csv", index=False)

In [ ]:
# Identify numerical columns
numerical_columns = classification_file.select_dtypes(include=['number']).columns

# Print the maximum value for each numerical column
for column in numerical_columns:
    max_value = classification_file[column].max()
    print(f"Maximum value in column '{column}': {max_value}")

In [26]:
# Load classification.csv (assuming it has a column named 'serial_no')
classification_file = pd.read_csv("../dataset/preprocessed/classification.csv")

# Filter the dataframe to include only rows with serial_no in valid_serials
classification_cleaned = classification_file[classification_file["serial_no"].isin(valid_serials)]

# Save the filtered data to classification_cleaned.csv
classification_cleaned.to_csv("../dataset/processed/classification_cleaned.csv", index=False)

print("Filtered classification data saved to ../dataset/processed/classification_cleaned.csv.")

C:\Users\vtngu\AppData\Local\Temp\ipykernel_45828\2158118902.py:2: DtypeWarning: Columns (0,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  classification_file = pd.read_csv("../dataset/preprocessed/classification.csv")


Filtered classification data saved to ../dataset/processed/classification_cleaned.csv.


In [25]:

# Load datasets
mark_cleaned = pd.read_csv("../dataset/processed/mark_cleaned.csv")  # Contains serial_no, mark_id_char
statement_cleaned = pd.read_csv("../dataset/processed/statement_cleaned.csv")  # Contains serial_no, statement_text
nice_class = pd.read_csv("../dataset/processed/nice_class.csv")  # Contains serial_no, nice_class_code, nice_class_description

# Merge the datasets on serial_no
merged_df = mark_cleaned.merge(statement_cleaned, on="serial_no", how="inner")
merged_df = merged_df.merge(nice_class, on="serial_no", how="inner")

# Create the formatted column
merged_df["formatted_text"] = merged_df.apply(
    lambda row: f"[cls] {row['mark_id_char']} [sep] {row['statement_text']} [sep] {row['nice_class_code']} [sep] {row['nice_class_description']} [cls]",
    axis=1
)

# Select only the required column
final_df = merged_df[["formatted_text"]]

# Save to CSV
output_file = "../dataset/processed/formatted_data.csv"
final_df.to_csv(output_file, index=False)

print(f"Formatted data saved to {output_file}.")


KeyError: 'serial_no'